In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Regression과 Classification 두 모델 이용해서 Bagging 실습

In [4]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [5]:
'''
<집 값 예측> - 'Boston house' data와는 다른 데이터

id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [14]:
ncar = data.shape[0] # 데이터 개수
nvar = data.shape[1] # 변수 개수
print('ncar : %d' % ncar, '\n', 'nvar : %d' % nvar)

ncar : 21613 
 nvar : 14


## 의미가 없다고 판단되는 변수 제거

In [17]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis=1)

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [18]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인
# 학습 데이터 : 약 15000개

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증

In [22]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant="add") # 상수항 추가
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

# R-squared : y의 총 변동성 분의 x가 설명하는 변동의 비율 -> 약 60%
# bathrooms 수 많으면 집값 올라감, bedrooms 수 많으면 집값 떨어짐

'''
기존 강사님 code

regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과
'''

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Sat, 21 Mar 2020   Prob (F-statistic):               0.00
Time:                        15:05:11   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
# 예측을 해보자
sm_test_x = sm.add_constant(test_x, has_constant="add") # test할 데이터만들기, test 데이터에도 상수항 추가
sm_model_predict = fitted_sm_model.predict(sm_test_x)

C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [25]:
sqrt(mean_squared_error(sm_model_predict, test_y)) # 하나의 모델만 사용했을 때 RMSE
# 굉장히 큰 값
# 이따가 다른 모델과 MSE값 비교해볼건데 지금 상태로는 비교하기 쉽지않아서 루트씌우자

239804.29670858168

## Bagging한 결과가 일반적인 결과보다 좋은지 확인

In [44]:
# Bagging을 할 수 있는 라이브러리 패키지가 있어서 그거 사용하면되는데
# 일단 처음이니까 for문 이용해서 앙상블하는 코드 작성해보자
# 변수를 좀 다르게 추출한다거나 다른 기법 사용하기 편하기때문에 직접 for문 사용하는 것도 좋다

bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 1부터 train_x 개수만큼 데이터 가져와
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터 복원추출 (추출할 변수:data_index, 추출할 요소의 개수:train_x.shape[0])
    print(len(set(random_data_index)))
    # set() : ex) set([1, 1, 3, 2, 4, 1]) = 1, 2, 3, 4 
    # unique한 것만 추출
    # 학습데이터 약 15000개 중 unique한 데이터는 약 9500~9600개 => 데이터 중 약 63%만 추출되는 게 맞지
    
    sm_train_x = train_x.iloc[random_data_index] # 복원추출 된 학습데이터 가져와서 다시 fitting시키는 과정 시행
    sm_train_y = train_y.iloc[random_data_index] # y도 똑같이 가져와야겠지
    sm_train_x = sm.add_constant(sm_train_x, has_constant="add")
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    pred = fitted_sm_model.predict(sm_test_x) # test할 데이터 위에서 만들었으므로 갖다쓰기
    bagging_predict_result.append(pred) # 결과값 append
    print(sqrt(mean_squared_error(pred, test_y)))
    
# MSE : 이전에 전체 데이터 다 쓴 경우보다 아주 조금 높게 나왔음

9511
241128.38173926383
9542


C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


239764.76895080705
9572
240452.83687778324
9562


C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


240799.3924695097
9545
239652.70723645116
9552
240651.52607320354
9507

C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)



239703.37733402164
9529
239947.3382701117

C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)



9535
239931.53957328343
9574
240369.15494944


C:\Users\hyun\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [45]:
bagging_predict_result[1] # 데이터 복원추출했을 때의 예측값

735      5.651988e+05
2830     7.255256e+05
4106     1.112184e+06
16218    1.479253e+06
19964    6.971432e+05
             ...     
12606    6.080202e+05
14393    6.916773e+05
6899     3.207711e+05
85       9.062189e+05
21363    4.294984e+05
Length: 6484, dtype: float64

In [46]:
# n(1~n)번째 리스트에 있는 첫번째 값들을 각각 다 더하고 평균을 내야 최종적인 ensemble 예측값 나오겠지
bagging_predict = []
for lst2_index in range(test_x.shape[0]): # test 데이터 개수만큼 반복
    # lst2_index : 데이터의 index
    temp_predict = [] # 임시 리스트
    for lst_index in range(len(bagging_predict_result)):
        # bagging_predict_result[1], bagging_predict_result[2], bagging_predict_result[3] 등등 데이터 가져와서 append-> 평균값
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))
    # for문 다 돌면 이젠 bagging_predict_result[1]의 두번째 값, bagging_predict_result[2]의 두번째 값
    # 등등 또 append 시켜서 평균값 구하는 거 반복

In [47]:
# 평균내서 predict 값 내기
sqrt(mean_squared_error(bagging_predict, test_y))
# for문 이용해서 bagging했을 때 ensenble 모델의 평균적인 RMSE
# 하나의 모델만 사용했을 때의 RMSE보다 증가했지 -> 오히려 더 안좋은 결과가 나왔음

239988.8077716596

## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증(Scikit-Learn)

In [48]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression()
linear_model1 = regression_model.fit(train_x, train_y)

## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [69]:
# 패키지 이용해서 bagging 실습
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators=5,# 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y)# 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239913.68717400832


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 그렇다면 Sampling을 많이 해보자!

In [68]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 239830.76688097368


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [70]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict_tree = tree_model.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict_tree, test_y)))) # RMSE 결과 (test 데이터에 대한)

# 성능이 굉장히 안좋아졌지
# 오히려 회귀모델 하나 적합시키는 게 훨씬 더 좋을 결과..

RMSE: 296114.43756303575


In [71]:
# 다시 for문 이용

bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index]
    sm_train_y = train_y.iloc[random_data_index] 
    decision_tree_model = DecisionTreeRegressor()
    tree_model = decision_tree_model.fit(sm_train_x, sm_train_y) 
    predict_tree = tree_model.predict(test_x)
    bagging_predict_result.append(predict_tree) 
    print(sqrt(mean_squared_error(predict_tree, test_y)))
    
# tree 모델은 overfitting이 심하게 일어나서 지금 좋은 모델이 하나도 없어
# 다 MSE값이 높잖아

9504
299722.1939446347
9578
301004.5606818526
9595
280100.290822849
9549
302763.75482187513
9536
305106.8934695088
9533
293515.2077458066
9615
297869.803692905
9576
290049.18113542284
9639
307084.0288660091
9593
271399.9676319777


In [88]:
bagging_predict = []
for lst2_index in range(test_x.shape[0]): # test 데이터 개수만큼 반복
    # lst2_index : 데이터의 index
    temp_predict = [] # 임시 리스트
    for lst_index in range(len(bagging_predict_result)):
        # bagging_predict_result[1], bagging_predict_result[2], bagging_predict_result[3] 등등 데이터 가져와서 append-> 평균값
        temp_predict.append(bagging_predict_result[lst_index][lst2_index])
        # 위 코드에서는 .values[]써서 value도 인덱스를 할당해서 가져왔어야했는데 이제는 array형태라 바로 [][]가능
    bagging_predict.append(np.mean(temp_predict))

In [90]:
sqrt(mean_squared_error(bagging_predict, test_y))
# 근데 모델들의 예측값을 평균냈더니 성능이 굉장히 좋아졌음
# 이게 앙상블의 기본적인 컨셉!
# 앙상블쓰면 성능 올라갈 확률이 높고, 그 base model은 overfitting이 잘 되는 게 일반적임
# 그 이유는 test data에 대한 다양한 의견을 수렴하기위해서

236514.25175656882

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [100]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 10, # 10번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 237378.2649133019


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 30번)

In [107]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링 (일반적으로 30번, 많이하면 50번)
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 230753.69555253806


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
